## Interview Query Question #11 | Employee Salaries

Good morning. Here's your sql question for today.

This question was asked by: Microsoft

employees table  

columns	types  
id	int  
first_name	varchar  
last_name	varchar  
salary	int  
department_id	int  
 

departments table  

columns	types  
id	int  
name	varchar   

1. Given the tables above, select the top 3 departments by the highest percentage of employees making over 100K in salary and have at least 10 employees.

Example output:

100K %	department name	number of employees  
90%	engineering	25  
50%	marketing	50  
12%	sales	12  

2. Let’s say due to an ETL error, the employee table instead of updating the salaries every year when doing compensation adjustments, did an insert instead. The head of HR still needs the current salary of each employee. Write a query to get the current salary for each employee.

Assume no duplicate combination of first and last names. (I.E. No two John Smiths)

In [ ]:
# idea: 100k+ salary employees, 10+ employee departments, left join, rank window function
# 100k% = 100k count / total count * 100

with sixfigures as (
select employee_id, salary, department_id
from employees 
where salary > 100000)
, 
bigdepts as (
select department_id, name, count(employee_id) as num_employees
from departments d join employees s on d.id = e.department_id
group by 1, 2
having num_employees >= 10)
,
raw as (
select name, count(e.employee_id) as num_all, count(s.employee_id) as num_sixfigs 
from employees e left join sixfigures s on e.employee_id = s.employee_id
where name in (select name from bigdepts)
group by 1
)

select (num_sixfigs / num_all) * 100 as "100k %", name as department_name, num_sixfigs as number_of_employees
from raw

In [ ]:
# idea: insert pushes new records into end of table and primary key autoincrements

with ranked as (
select concat(first_name,  last_name) as name, employee_id, salary, (rownumber() over employee_id partition by name order by employee_id desc) as row
from employees)

select *
from ranked
where row = 1